## HH парсер

### условия:
* санкт-петербург
* другие города, если удаленка
* ds/bi/аналитик
* sql | python | powerbi 
* за текущий день
* не middle | не teamlead | не старший аналитик | не ведущий аналитик
* вакансии, на которые я еще не отклилася
* показывать True,если есть контактные данные

In [1]:
import requests, json, os
import pandas as pd
import numpy as np
import warnings
from datetime import datetime as dt
warnings.filterwarnings('ignore')
import webbrowser
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict
from tqdm import tqdm_notebook
from datetime import datetime as dt
import time
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('seaborn-ticks')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from bs4 import BeautifulSoup 
from tqdm import tqdm_notebook
from wordcloud import WordCloud, STOPWORDS

In [2]:
def collect_hh_vacancies(area, query, pages):    
    #######################################################################################
    def getPage(area, page, query):
        """
        Создаем метод для получения страницы со списком вакансий.
        Аргументы:
            page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
        """

        # Справочник для параметров GET-запроса
        params = {
            'text': f'NAME:{query}', 
            'area': f'{area}', 
            'page': page, 
            'per_page': 100
            }
        req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
        data = req.content.decode() 
        req.close()
        return data

    def flatten_json(json_file):
        L= []
        for k, v in i.items():
            if isinstance(v, dict):
                for k2, v2 in v.items():
                    if not(isinstance(v2, dict)):
                        L.append((f'{k}.{k2}', v2))
                    else:
                        for k3, v3 in v2.items():
                            L.append((f'{k}.{k2}.{k3}', v3))
                        
            else:
                L.append((k, v))        
        _df = pd.DataFrame.from_records(L).set_index(0)
        _df.columns = _df.loc['id'].values    
        return _df.iloc[1:].T  
    ###########################################################################################
    
    # список с коллекцией выгруженных вакансий
    L_rows=[]    

    for page_idx in tqdm_notebook(pages):

        # выгружаем
        json_spb = json.loads(getPage(area=area, page=page_idx,query=query))     
        
        try:
            
            for i in json_spb['items']: 
                
                # коллекционируем
                L_rows.append(flatten_json(i))
                
                # спим, чтобы сайт не ограничил доступ
                time.sleep(.1)   
        except:
            pass  
    
    # собираем датафрейм
    df = pd.concat(L_rows)
    for col in df.columns:
        try:
            df[col] = df[col].astype(np.float32)
        except:
            pass
    df['published_at'] = pd.to_datetime(df['published_at'])
    return df.sort_values('published_at', ascending = False).reset_index(drop = True)

In [3]:
df1 = collect_hh_vacancies(area=113,query='аналитик', pages=np.arange(10))
df2 = collect_hh_vacancies(area=113,query='data scientist', pages=np.arange(10))
df3 = collect_hh_vacancies(area=113,query='bi', pages=np.arange(10))

In [32]:
# собираем датафрейм
df = pd.concat([df1, df2, df3], 0).reset_index(drop = True).dropna(subset = ['snippet.requirement'])

# требования
req = df['snippet.requirement'].str.lower()
mask_spb = df['area.name'].isin(['Санкт-Петербург'])
mask_udalenka = df['schedule.name'] == 'Удаленная работа'
mask_python = req.apply(lambda x: 'python' in x)
mask_sql = req.apply(lambda x: 'sql' in x)
mask_bi = req.apply(lambda x: 'bi' in x)

# используем колонки
cols_to_use = ['name', 'published_at', 'salary.from', 'salary.to',\
               'employer.name', 'snippet.requirement',\
               'snippet.responsibility', 'schedule.name', 'alternate_url']
ru_cols_to_use=['название', 'дата_публикации', 'зп_от', 'зп_до', 'компания', 'требования',\
                'обязанности', 'график_работы', 'url']

# финальная таблица(вакансии за последний день)
df_final = df[(mask_spb|(~mask_spb&mask_udalenka)) & (mask_python|mask_sql|mask_bi)]\
               .reset_index(drop = True)\
               [cols_to_use]\
               .rename(columns = dict(zip(cols_to_use, ru_cols_to_use)))
df_final = df_final[df_final['зп_от'].apply(lambda x: np.isnan(x)|(x<100000))]

# дата публикации
datetimes = pd.to_datetime(df_final['дата_публикации'].dt.date)
# последняя публикация
date_today = pd.to_datetime(dt.now().date())
df_final = df_final[((date_today - datetimes) / np.timedelta64(1, 'D'))<=1]                   
df_final = df_final[~df_final.duplicated()]          

# ранжирование вакансий по требованиям
req = df_final['требования'].str.lower()
sql_mask = req.str.contains('sql').astype(int)
pandas_mask = req.str.contains('pandas').astype(int)
python_mask = req.str.contains('python').astype(int)
df_final = df_final.loc[(python_mask+pandas_mask+sql_mask).sort_values(ascending = False).index]

# выбрасываем middle и senior
mask_notmiddle = df_final['название'].str.lower().apply(lambda x: 'middle' not in x)
mask_notsenior = df_final['название'].str.lower().apply(lambda x: 'senior' not in x)
mask_notsenior_v2 = df_final['название'].str.lower().apply(lambda x: 'старший' not in x)
mask_notsenior_v3 = df_final['название'].str.lower().apply(lambda x: 'ведущий' not in x)

df_final = df_final[(mask_notmiddle)&(mask_notsenior)&(mask_notsenior_v2)&(mask_notsenior_v3)].reset_index(drop = True)   


df_today = df_final[~df_final['компания'].isin(['Филберт', 'Softwise'])]

# проверяем, отликались ли мы уже на вакансию
urls_to_use = []
contacts_flag = []
for url in df_today['url'].values:
    # открываем страницу в браузере
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    if 'Откликнуться' in str(soup):
        urls_to_use.append(url)
    if 'Показать контакты' in str(soup):
        contacts_flag.append(True)    
    driver.close()

In [39]:
mask = df_today['url'].isin(urls_to_use)
df_actual = df_today[mask]
df_actual['есть_контактные_данные'] = pd.Series(contacts_flag).loc[mask]
df_actual = df_actual.fillna(False)
df_final = df_actual[['название', 'дата_публикации', 'требования', 'обязанности',\
                      'график_работы', 'url', 'есть_контактные_данные']]

In [41]:
df_final

,название,дата_публикации,требования,обязанности,график_работы,url,есть_контактные_данные
0,Data Analyst,2021-06-20 18:44:45+03:00,Опыт аналитики данных от двух лет. Глубокое по...,"Esports Charts - украинская ИТ-компания, разра...",Удаленная работа,https://hh.ru/vacancy/44957591,True
1,Data Analyst/ Аналитик в Big Data,2021-06-20 14:05:07+03:00,"Высшее образование (математика, бизнес-информа...",Помогать заказчикам уточнять их пожелания к ре...,Полный день,https://hh.ru/vacancy/44676264,True
2,Бизнес-аналитик,2021-06-20 23:46:47+03:00,Образование высшее математическое или IT. Опыт...,Внедрение CRM системы. Построение системы и ан...,Полный день,https://hh.ru/vacancy/44527353,True
3,Аналитик (удаленно),2021-06-20 18:18:29+03:00,Аналитический склад ума. Законченное высшее те...,"Аналитика бизнес-процессов, интеграционных реш...",Удаленная работа,https://hh.ru/vacancy/44956879,True
4,Системный аналитик,2021-06-20 15:34:41+03:00,"Опыт работы с SQL, опыт проектирования REST AP...",Проведение системного анализа разрабатываемого...,Удаленная работа,https://hh.ru/vacancy/45537155,True
5,Системный аналитик,2021-06-20 14:14:50+03:00,Высшее техническое/математическое образование....,Изучить предметную область (фрод) и функционал...,Гибкий график,https://hh.ru/vacancy/44945902,True
6,Аналитик /Data Analyst,2021-06-20 12:10:07+03:00,"Отличная математическая подготовка, знание ста...",Предстоит заниматься аналитической поддержкой ...,Полный день,https://hh.ru/vacancy/41124052,True
7,Дата Аналитик,2021-06-21 06:37:02+03:00,Опыт работы с большими объемами данных - обяза...,"Поиск источников данных. Сбор, обработка, сист...",Удаленная работа,https://hh.ru/vacancy/44739112,True
8,Risk Analyst/Data Scientist,2021-06-20 20:50:47+03:00,You love working with data: mining insights an...,VIA SMS Group is an alternative financial serv...,Удаленная работа,https://hh.ru/vacancy/44959404,True
9,Data Scientist,2021-06-20 17:02:12+03:00,"Python, Vertica, Airflow, Git, Docker, Grafana...",Отвечать за полный цикл внедрения алгоритмов -...,Полный день,https://hh.ru/vacancy/39307094,True
